In [1]:
import pandas as pd
import numpy as np

timeLong = 30
step = 20
np.random.seed(2016)

In [2]:
logPathSortUniq = '/Users/holazhai/Documents/workspace/readshift/result_uniq_sort_ha_holaverse_int.log'
colnameSortUniq = ['created_day', 'pdtid', 'pid','u','dt']
dfSortUniq = pd.read_table(logPathSortUniq,sep='\t',names=colnameSortUniq)
dfSortUniq['key'] = zip(dfSortUniq['created_day'],dfSortUniq['pdtid'],dfSortUniq['pid'],dfSortUniq['u'])

In [3]:
def getDfDic(dfSortUniq):
    dfDic = {}
    for index, row in dfSortUniq.iterrows():
        key = row['key']
        value = row['dt']
        #print key,value
        if dfDic.has_key(key):
            dfDic[key].append(value)
        else:
            dfDic[key]=[value]
    return dfDic
dfDic=getDfDic(dfSortUniq)

import datetime
def plusDay(strTime):
    time1 = datetime.datetime.strptime(strTime,'%Y-%m-%d')
    next_dat = time1 + datetime.timedelta(hours = 24)
    next_str = datetime.datetime.strftime(next_dat,'%Y-%m-%d')
    return str(next_str)

def diffDay(day1,day2):
    d1=datetime.datetime.strptime(day1,'%Y-%m-%d')
    d2=datetime.datetime.strptime(day2,'%Y-%m-%d')
    return (d1-d2).days

In [4]:
def getDfListKeyAndValue(dfDic):
    dfListKey = []
    dfListValue = []
    for key in dfDic.keys():
        l = dfDic[key]
        ltempKey = []
        ltempValue = []
        times = key[0]
    
        if times in l:
            l.remove(times)
        
        if len(l)==0:
            pass;
        else:
            #print l,times,plusDay(times),diffDay(plusDay(times),times)
            ltempKey.append(key)
            for i in range(len(l)):
                diff = diffDay(l[i],times)
                times = l[i]
                for i in range(diff-1):
                    ltempValue.append(0)
                ltempValue.append(1)
            if len(ltempValue)<timeLong:
                for i in range(timeLong-len(ltempValue)):
                    ltempValue.append(0)
            #print ltempKey,ltempValue
            dfListKey.append(ltempKey)
            dfListValue.append(ltempValue)
            
    return dfListKey,dfListValue
dfListKey,dfListValue = getDfListKeyAndValue(dfDic)
            
print len(dfListValue)
print len(dfListKey)

16724
16724


In [5]:
dfArrayKey=[]
dfArrayValue =[]
length = len(dfListKey)

for i in range(length):
    for j in range(timeLong-step-1):
        #取五分之一的负样本,为了样本平衡
        if dfListValue[i][j+step] == 0:
            if np.random.random() >0.80:
                dfArrayValue.append(dfListValue[i][j:j+step+1])
                dfArrayKey.append(dfListKey[i])
        else:
            dfArrayValue.append(dfListValue[i][j:j+step+1])
            dfArrayKey.append(dfListKey[i])
    

In [6]:
x_train = np.array(dfArrayValue)[:,0:step]
y_train = np.array(dfArrayValue)[:,step] 
x_train = x_train.reshape(x_train.shape[0],1,step)
print x_train.shape
print y_train.shape

(49749, 1, 20)
(49749,)


In [7]:
#train NN
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.optimizers import Adam
from keras.layers.recurrent import LSTM

import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


Using TensorFlow backend.


In [8]:
model = Sequential()
model.add(LSTM(32,input_shape=(1,step)))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='relu'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
print('Train...')
model.fit(x_train, y_train, batch_size=64, nb_epoch=5)

Train...
Epoch 1/5
49749/49749 [==============================] - 15s - loss: 0.3998 - acc: 0.8474    
Epoch 2/5
49749/49749 [==============================] - 16s - loss: 0.3278 - acc: 0.8727    
Epoch 3/5
49749/49749 [==============================] - 16s - loss: 0.3060 - acc: 0.8895    
Epoch 4/5
49749/49749 [==============================] - 17s - loss: 0.2993 - acc: 0.8944    
Epoch 5/5
49749/49749 [==============================] - 16s - loss: 0.3167 - acc: 0.8844    


In [10]:
#存在问题，正负样本数目不一致
y_train
a = np.array(filter(lambda x: x > 0, y_train))
print len(a)
print len(y_train)

24647
49749


In [11]:
x1=x_train[0:-5000][:][:]
print x1.shape
x2=x_train[-5000:][:][:]
print x2.shape
y1=y_train[0:-5000][:]
print y1.shape
y2=y_train[-5000:][:]
print y2.shape

(44749, 1, 20)
(5000, 1, 20)
(44749,)
(5000,)


In [12]:
model2 = Sequential()
model2.add(LSTM(64,input_shape=(1,step),return_sequences=True))
model2.add(LSTM(32))
model2.add(Dense(32,activation='relu'))
model2.add(Dense(1,activation='relu'))

model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [13]:
print('Train...')
model2.fit(x1, y1, batch_size=32, nb_epoch=5,shuffle=True,validation_data=(x2, y2))

Train...
Train on 44749 samples, validate on 5000 samples
Epoch 1/5
44749/44749 [==============================] - 54s - loss: 0.4203 - acc: 0.8423 - val_loss: 0.3402 - val_acc: 0.8894
Epoch 2/5
44749/44749 [==============================] - 50s - loss: 0.5438 - acc: 0.8430 - val_loss: 0.2945 - val_acc: 0.8912
Epoch 3/5
44749/44749 [==============================] - 49s - loss: 0.2858 - acc: 0.8922 - val_loss: 0.2774 - val_acc: 0.8996
Epoch 4/5
44749/44749 [==============================] - 46s - loss: 0.3517 - acc: 0.8671 - val_loss: 0.5996 - val_acc: 0.5088
Epoch 5/5
44749/44749 [==============================] - 48s - loss: 0.3256 - acc: 0.8642 - val_loss: 0.2800 - val_acc: 0.9014


In [14]:
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
y2_p=model2.predict(x2)
testScore = math.sqrt(mean_squared_error(y2, y2_p))
print testScore

0.282043108286


In [15]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
#LSTM For Regression with Time Steps
xx1=x1.reshape(x1.shape[0],x1.shape[2],x1.shape[1])
print xx1.shape
yy1=y1
xx2=x2.reshape(x2.shape[0],x2.shape[2],x2.shape[1])
yy2=y2
print xx2.shape

model3 = Sequential()
model3.add(LSTM(64,input_shape=(step,1),return_sequences=True))
model3.add(LSTM(64))
model3.add(Dense(32,activation='tanh'))
model3.add(Dense(1,activation='tanh'))

model3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

(44749, 20, 1)
(5000, 20, 1)


In [ ]:
print('Train...')
model3.fit(xx1, yy1, batch_size=64, nb_epoch=20,shuffle=True,validation_data=(xx2, yy2))

Train...


In [ ]:
#LSTM With Memory Between Batches
#http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
#验证留存率计算结果
#2016-07-24600004ha_holaverse_int 28742
print len(dfArrayKey)
print type(dfArrayKey)
print len(dfArrayValue)
print type(dfArrayValue)
#选择测试数据 2016-06-06#600001#Organic_up

print type(dfListKey)
print dfListKey[0][0]
print dfListValue[0]
print len(dfListValue)


In [ ]:
dfArrayKeyTest=[]
dfArrayValueTest =[]
length = len(dfListKey)
for i in range(length):
    keyTimes = dfListKey[i][0][0]
    proid = dfListKey[i][0][1]
    pid = dfListKey[i][0][2]
    if keyTimes=='2016-07-25' and proid==600004 and pid=='ha_holaverse_int':
        dfArrayValueTest.append(dfListValue[i][0:step+1])
        dfArrayKeyTest.append(dfListKey[i])


In [ ]:
testX = np.array(dfArrayValueTest)[:,0:step]
testY = np.array(dfArrayValueTest)[:,step] 
print testX.shape
testX = testX.reshape(testX.shape[0],step,1)
print testX.shape

In [ ]:
predictTestX = model3.predict(testX)
pos = np.where(predictTestX>0.8)
print len(pos[0])
print sum(testY)

In [ ]:
predictTestX = model.predict(testX.reshape(testX.shape[0],testX.shape[2],testX.shape[1]))
pos = np.where(predictTestX>0.8)
print len(pos[0])
print sum(testY)

In [ ]:
predictTestX = model2.predict(testX.reshape(testX.shape[0],testX.shape[2],testX.shape[1]))
pos = np.where(predictTestX>0.8)
print len(pos[0])
print sum(testY)

In [ ]:
#测试过程
logPathSortUniq20160801 = '/Users/holazhai/Documents/workspace/readshift/result_uniq_sort_20160801.log'
colnameSortUniq20160801 = ['created_day', 'pdtid', 'pid','u','dt']
dfSortUniq20160801 = pd.read_table(logPathSortUniq20160801,sep='\t',names=colnameSortUniq20160801)

dfSortUniq20160801['key'] = zip(dfSortUniq20160801['created_day'],dfSortUniq20160801['pdtid'],dfSortUniq20160801['pid'],dfSortUniq20160801['u'])

In [ ]:
dfDic20160801=getDfDic(dfSortUniq20160801)
dfListKey20160801,dfListValue20160801 = getDfListKeyAndValue(dfDic20160801)
print len(dfListKey20160801)
print len(dfListValue20160801)

In [ ]:
dfArrayKeyTest20160801=[]
dfArrayValueTest20160801 =[]
length = len(dfListKey20160801)
print length
for i in range(length):
    keyTimes = dfListKey20160801[i][0][0]
    proid = dfListKey20160801[i][0][1]
    pid = dfListKey20160801[i][0][2]
    if keyTimes=='2016-08-04' and proid==600004 and pid=='ha_holaverse_int':
        #print dfListValue20160801[i][0:21]
        dfArrayValueTest20160801.append(dfListValue20160801[i][0:21])
        dfArrayKeyTest20160801.append(dfListKey20160801[i])

In [ ]:
testX = np.array(dfArrayValueTest20160801)[:,0:step]
testY = np.array(dfArrayValueTest20160801)[:,step] 
print testX.shape
testX = testX.reshape(testX.shape[0],step,1)
print testX.shape
predictTestX = model3.predict(testX)
pos = np.where(predictTestX>0.8)
print "model3:"
print len(pos[0])
print sum(testY)
predictTestX = model.predict(testX.reshape(testX.shape[0],testX.shape[2],testX.shape[1]))
pos = np.where(predictTestX>0.8)
print "model:"
print len(pos[0])
print sum(testY)
predictTestX = model2.predict(testX.reshape(testX.shape[0],testX.shape[2],testX.shape[1]))
pos = np.where(predictTestX>0.8)
print "model2:"
print len(pos[0])
print sum(testY)